In [1]:
import os
import io
import numpy as np
import multiprocessing as mp
import shutil
import subprocess

from pymatgen.ext.matproj import MPRester
from pymatgen.core import Structure, Composition, Element

In [2]:
with open(os.path.expanduser("~/.mpkey.txt"), "r+") as file:
    apikey = file.readline()
rester = MPRester(apikey)
struct1 = rester.get_structure_by_material_id("mp-2133", final = False, conventional_unit_cell=True)
struct2 = rester.get_structure_by_material_id("mp-1986", final = False, conventional_unit_cell=True)

In [3]:
vcomp1 = Composition(struct1.formula)
vcomp2 = Composition(struct2.formula)

pps = ["Zn.pbe.upf", "O.pbe.upf"]

In [4]:
#cell_params
#struct1
bio = io.BytesIO()
np.savetxt(bio, struct1.lattice.matrix, fmt="%0.8f")# to much precision in the input files causes convergence issues. to little causes symmetry issues.
cell_parameters_block1 = bio.getvalue().decode('latin1')
#struct2
np.savetxt(bio, struct2.lattice.matrix, fmt="%0.8f")# to much precision in the input files causes convergence issues. to little causes symmetry issues.
cell_parameters_block2 = bio.getvalue().decode('latin1')

In [5]:
#prepare ordered species 

# extract element key from pps
pppart_list = []
for pp in pps:
    ppparts = pp.split('.')
    pppart_list.append(ppparts[0])
    
#sort formula elements and atomic mass by decending atomic mass
amass_ar = np.array([Element(el).atomic_mass for el in vcomp1.to_reduced_dict.keys()])
massive_first = np.argsort(amass_ar).tolist()[::-1]
amass_falling = amass_ar[massive_first]
el_falling = np.array([el for el in vcomp1.to_reduced_dict.keys()])[massive_first]

#sort pps to match
pps_amass_ar = np.array([Element(el).atomic_mass for el in pppart_list])
pps_massive_first = np.argsort(pps_amass_ar).tolist()[::-1]
pps_falling = np.array(pps)[pps_massive_first]


In [6]:
#atomic species
atomic_species_block1 = """"""
for pp, amass, species in zip(pps_falling, amass_falling, el_falling):
    block_line = f"{species} {amass} {pp}\n"
    atomic_species_block1 += block_line
    
atomic_species_block2 = """"""
for pp, amass, species in zip(pps_falling, amass_falling, el_falling):
    block_line = f"{species} {amass} {pp}\n"
    atomic_species_block2 += block_line

In [7]:
#atomic positions
def get_fractional_coords_as_qe_array(periodicsitesobj):
    at_site = np.array([str(*periodicsitesobj.species.to_reduced_dict.keys()), periodicsitesobj.a, periodicsitesobj.b, periodicsitesobj.c])
    # str might break if the species has whitespaces in it... I think. Or maybe that's just for unpacked lists
    return at_site

atomic_positions_block1 = np.array(list(map(lambda x: get_fractional_coords_as_qe_array(x), struct1.sites)))
bio = io.BytesIO()
np.savetxt(bio, atomic_positions_block1, fmt="%s", encoding="latin1")
atomic_positions_block1 = bio.getvalue().decode('latin1')

atomic_positions_block2 = np.array(list(map(lambda x: get_fractional_coords_as_qe_array(x), struct2.sites)))
bio = io.BytesIO()
np.savetxt(bio, atomic_positions_block2, fmt="%s", encoding="latin1")
atomic_positions_block2 = bio.getvalue().decode('latin1')

In [8]:
vcrelax_input1 = f"""
 &control
    calculation='vc-relax',
    restart_mode='from_scratch',
    prefix='{vcomp1.reduced_formula}',
    outdir='./',
    pseudo_dir = './simtool/pseudo/',
    etot_conv_thr=1.0d-6,
    forc_conv_thr=1.0d-6,
    nstep=100,
/
 &system    
    ibrav= 0, celldm(1)=1 ,nat= 4, ntyp= 2,
    ecutwfc = 50, ecutrho = 200,
    occupations="fixed",
    nosym = TRUE,
/
 &electrons
    mixing_beta =0.7,
    conv_thr =1.0d-8,
/
 &ions
    ion_dynamics='bfgs'
/
&cell
    cell_dynamics='bfgs',
    press=0.0,
    cell_factor=4.0,
    press_conv_thr=0.5,
/
CELL_PARAMETERS (alat= 1.00000000)
{cell_parameters_block1}
ATOMIC_SPECIES
{atomic_species_block1}
ATOMIC_POSITIONS (crystal)
{atomic_positions_block1}
K_POINTS (automatic)
3 2 2 0 0 0
"""

vcrelax_input2 = f"""
 &control
    calculation='vc-relax',
    restart_mode='from_scratch',
    prefix='{vcomp2.reduced_formula}',
    outdir='./',
    pseudo_dir = './simtool/pseudo/',
    etot_conv_thr=1.0d-6,
    forc_conv_thr=1.0d-6,
    nstep=100,
/
 &system    
    ibrav= 0, celldm(1)=1 ,nat= 4, ntyp= 2,
    ecutwfc = 50, ecutrho = 200,
    occupations="fixed",
    nosym = TRUE,
/
 &electrons
    mixing_beta =0.7,
    conv_thr =1.0d-8,
/
 &ions
    ion_dynamics='bfgs'
/
&cell
    cell_dynamics='bfgs',
    press=0.0,
    cell_factor=4.0,
    press_conv_thr=0.5,
/
CELL_PARAMETERS (alat= 1.00000000)
{cell_parameters_block2}
ATOMIC_SPECIES
{atomic_species_block2}
ATOMIC_POSITIONS (crystal)
{atomic_positions_block2}
K_POINTS (automatic)
3 2 2 0 0 0
"""

In [9]:
vcrelax_input1

'\n &control\n    calculation=\'vc-relax\',\n    restart_mode=\'from_scratch\',\n    prefix=\'ZnO\',\n    outdir=\'./\',\n    pseudo_dir = \'./simtool/pseudo/\',\n    etot_conv_thr=1.0d-6,\n    forc_conv_thr=1.0d-6,\n    nstep=100,\n/\n &system    \n    ibrav= 0, celldm(1)=1 ,nat= 4, ntyp= 2,\n    ecutwfc = 50, ecutrho = 200,\n    occupations="fixed",\n    nosym = TRUE,\n/\n &electrons\n    mixing_beta =0.7,\n    conv_thr =1.0d-8,\n/\n &ions\n    ion_dynamics=\'bfgs\'\n/\n&cell\n    cell_dynamics=\'bfgs\',\n    press=0.0,\n    cell_factor=4.0,\n    press_conv_thr=0.5,\n/\nCELL_PARAMETERS (alat= 1.00000000)\n1.59205000 -2.75751149 0.00000000\n1.59205000 2.75751149 0.00000000\n0.00000000 0.00000000 5.15510000\n\nATOMIC_SPECIES\nZn 65.409 Zn.pbe.upf\nO 15.9994 O.pbe.upf\n\nATOMIC_POSITIONS (crystal)\nZn 0.6666666666666666 0.3333333333333333 0.5\nZn 0.3333333333333333 0.6666666666666666 0.0\nO 0.6666666666666666 0.3333333333333333 0.8803099999999999\nO 0.3333333333333333 0.6666666666666666

In [10]:
vcr_input_file1 = open(f"{vcomp1.reduced_formula}.vc-relax1.in", "w")
vcr_input_file1.write(vcrelax_input1)
vcr_input_file1.close()

vcr_input_file2 = open(f"{vcomp2.reduced_formula}.vc-relax2.in", "w")
vcr_input_file2.write(vcrelax_input2)
vcr_input_file2.close()

vcr_output_file1 = open(f"{vcomp1.reduced_formula}.vc-relax1.out", "w")
vcr_output_file2 = open(f"{vcomp2.reduced_formula}.vc-relax2.out", "w")

In [11]:
pp_args = ""
for pp in pps:
    pp_args += f"-i pseudo/{pp} "

In [12]:
subargs1 = ("4", "01:00:00", vcomp1, vcr_output_file1, vcr_input_file1, pp_args, "1")
subargs2 = ("4", "01:00:00", vcomp2, vcr_output_file2, vcr_input_file2, pp_args, "2")

def vcrsubmit(numnodes, walltime, vcomp, vcr_output_file, vcr_input_file, pp_args, run_num):
    COMMANDvcr = f"espresso-6.8_pw > {vcr_output_file.name}"
    SUBMITvcr = f"submit -n {numnodes} -w {walltime} -e QE_DISABLE_GGA_PBE=0 --runName {vcomp.reduced_formula}{run_num}vcr {COMMANDvcr} {pp_args} -i {vcr_input_file.name} "
    spvcr = subprocess.run(SUBMITvcr.split(), capture_output=True, text=True)
    vcr_output_file.close()

In [13]:
p1 = mp.Process(target=vcrsubmit, args=subargs1)
p2 = mp.Process(target=vcrsubmit, args=subargs2)

In [14]:
p1.run() #submit command cannot pull the necessary pps to the cluster?
p2.run()